## データセットの準備

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
def space_clean(row):
    row = row.replace(". ", "").replace(" .", "").replace("  ", " ")
    return row


def canonicalize(smiles):
    try:
        new_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles), canonical=True)
    except:
        new_smiles = None
    return new_smiles

In [3]:
df = pd.read_csv("./data/inchi_23l_reaction_t5_ready.csv")

In [4]:
# 必須カラムの存在チェックと補完
required_cols = ["REACTANT", "CATALYST", "REAGENT", "SOLVENT", "PRODUCT"]
for col in required_cols:
    if col not in df.columns:
        df[col] = ""

# 必要に応じてYIELDを標準化（0-1に正規化）
if "YIELD" in df.columns and df["YIELD"].max() >= 100:
    df["YIELD"] = df["YIELD"].clip(0, 100) / 100
else:
    df["YIELD"] = None

In [5]:
for col in ["REAGENT", "REACTANT", "PRODUCT"]:
    df[col] = df[col].apply(space_clean)
    df[col] = df[col].apply(lambda x: canonicalize(x) if x != " " else " ")
    df = df[~df[col].isna()].reset_index(drop=True)
    df[col] = df[col].apply(lambda x: ".".join(sorted(x.split("."))))

In [6]:
df["REAGENT"] = df["CATALYST"].fillna(" ") + "." + df["REAGENT"].fillna(" ")

In [7]:
df = df.loc[df[["YIELD"]].drop_duplicates().index].reset_index(drop=True)

## モデルの読み込み

In [8]:
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, AutoConfig, PreTrainedModel

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

/root/ReactionT5-bo-yield/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ReactionT5Yield(PreTrainedModel):
    config_class  = AutoConfig
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.model = T5ForConditionalGeneration.from_pretrained(self.config._name_or_path)
        self.model.resize_token_embeddings(self.config.vocab_size)
        self.fc1 = nn.Linear(self.config.hidden_size, self.config.hidden_size//2)
        self.fc2 = nn.Linear(self.config.hidden_size, self.config.hidden_size//2)
        self.fc3 = nn.Linear(self.config.hidden_size//2*2, self.config.hidden_size)
        self.fc4 = nn.Linear(self.config.hidden_size, self.config.hidden_size)
        self.fc5 = nn.Linear(self.config.hidden_size, 1)

        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
        self._init_weights(self.fc3)
        self._init_weights(self.fc4)
        self._init_weights(self.fc5)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.01)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.01)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, inputs):
        device = inputs['input_ids'].device

        with torch.no_grad():
            encoder_outputs = self.model.encoder(
                input_ids=inputs['input_ids'],
                attention_mask=inputs.get('attention_mask', None),
            )
            encoder_hidden_states = encoder_outputs[0]  # (B, L, H)

            dec_input_ids = torch.full(
                (inputs['input_ids'].size(0), 1),
                self.config.decoder_start_token_id,
                dtype=torch.long,
                device=device,
            )

            outputs = self.model.decoder(
                input_ids=dec_input_ids,
                encoder_hidden_states=encoder_hidden_states,
            )
            last_hidden_states = outputs[0]  # (B, 1, H)

        output1 = self.fc1(last_hidden_states.view(-1, self.config.hidden_size))
        output2 = self.fc2(encoder_hidden_states[:, 0, :].view(-1, self.config.hidden_size))
        output = self.fc3(torch.hstack((output1, output2)))
        output = self.fc4(output)
        output = self.fc5(output)
        return output * 100

In [10]:
# 収率予測（スカラー出力）
yield_tokenizer = AutoTokenizer.from_pretrained("sagawa/ReactionT5v2-yield")
yield_model = ReactionT5Yield.from_pretrained("sagawa/ReactionT5v2-yield")

In [11]:
# Dropoutレイヤーを学習モードに設定する関数
def enable_dropout(model):
    for module in model.modules():
        if isinstance(module, nn.Dropout):
            module.train()

In [12]:
# MC Dropoutによる推論
def mc_dropout_inference(model, tokenizer, reaction_smiles, n_samples=100, max_length=512):
    model.eval()  # まずモデル全体を評価モードに
    enable_dropout(model)  # Dropoutレイヤーのみ学習モードに
    
    device = next(model.parameters()).device
    
    # 単一の文字列の場合リストに変換
    if isinstance(reaction_smiles, str):
        reaction_smiles = [reaction_smiles]
    
    predictions = []
    
    with torch.no_grad():
        for _ in range(n_samples):
            batch_predictions = []
            
            for smiles in reaction_smiles:
                # トークン化
                inputs = tokenizer(
                    smiles, 
                    return_tensors="pt", 
                    max_length=max_length,
                    padding=True,
                    truncation=True
                ).to(device)
                
                # 予測実行
                output = model(inputs)
                batch_predictions.append(output.cpu().numpy().flatten())
            
            predictions.append(np.array(batch_predictions))
    
    # 結果を配列に変換: (n_samples, n_reactions)
    predictions = np.array(predictions)
    
    # 統計量計算
    mean_pred = np.mean(predictions, axis=0)
    var_pred = np.var(predictions, axis=0)
    
    if len(reaction_smiles) == 1:
        return {
            'mean': float(mean_pred[0]),
            'variance': float(var_pred[0])
        }
    else:
        return {
            'mean': mean_pred.tolist(),
            'variance': var_pred.tolist()
        }

## ベイズ最適化

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

In [14]:
# 辞書作成関数
def create_reaction_dictionaries(df):
    """
    データセットから反応辞書を作成
    
    Returns:
        tuple: (reactant_list, reagent_list, product_list, product_dict, true_yield_dict)
    """
    reactant_list = sorted(df["REACTANT"].unique())
    reagent_list = sorted(df["REAGENT"].unique())
    product_list = sorted(df["PRODUCT"].unique())
    
    # (reactant, reagent) -> product のマッピング
    product_dict = {
        (row["REACTANT"], row["REAGENT"]): row["PRODUCT"]
        for _, row in df.iterrows()
    }
    
    # (reactant, reagent, product) -> yield のマッピング
    true_yield_dict = {
        (row["REACTANT"], row["REAGENT"], row["PRODUCT"]): row["YIELD"]
        for _, row in df.iterrows()
    }
    
    print(f"Reactant candidates: {len(reactant_list)}")
    print(f"Reagent candidates: {len(reagent_list)}")
    print(f"Product candidates: {len(product_list)}")
    print(f"Known combinations: {len(product_dict)}")
    
    return reactant_list, reagent_list, product_list, product_dict, true_yield_dict

In [15]:
class BayesianOptimizationWithMCDropout:
    def __init__(self, model, tokenizer, reactant_list, reagent_list, product_dict, true_yield_dict, n_mc_samples=50):
        """
        MC Dropoutを使用したベイズ最適化（reactant-reagent組み合わせ探索）
        
        Args:
            model: 学習済みReactionT5Yieldモデル
            tokenizer: トークナイザー
            reactant_list: reactantの候補リスト
            reagent_list: reagentの候補リスト
            product_dict: (reactant, reagent) -> product のマッピング辞書
            true_yield_dict: (reactant, reagent, product) -> yield のマッピング辞書
            n_mc_samples: MC Dropoutのサンプル数
        """
        self.model = model
        self.tokenizer = tokenizer
        self.reactant_list = reactant_list
        self.reagent_list = reagent_list
        self.product_dict = product_dict
        self.true_yield_dict = true_yield_dict
        self.n_mc_samples = n_mc_samples
        
        # 全ての有効な組み合わせを事前計算
        self.valid_combinations = []
        for reactant in reactant_list:
            for reagent in reagent_list:
                if (reactant, reagent) in product_dict:
                    self.valid_combinations.append((reactant, reagent))
        
        print(f"有効な組み合わせ数: {len(self.valid_combinations)}")
        
        # 実験結果の記録
        self.experiment_history = []
        self.tried_combinations = set()
    
    def _predict_yield_with_uncertainty(self, reaction_smiles):
        """MC Dropoutによる収率予測"""
        result = mc_dropout_inference(
            self.model, self.tokenizer, reaction_smiles, 
            n_samples=self.n_mc_samples
        )
        return result['mean'], result['variance']
    
    def _acquisition_function(self, mean, variance, best_observed_yield, xi=0.01):
        """期待改善量（Expected Improvement）による獲得関数"""
        if variance <= 0:
            return 0.0
        
        std = np.sqrt(variance)
        z = (mean - best_observed_yield - xi) / std
        ei = (mean - best_observed_yield - xi) * norm.cdf(z) + std * norm.pdf(z)
        return ei
    
    def _get_best_observed_yield(self):
        """これまでの最良収率を取得"""
        if len(self.experiment_history) > 0:
            return max([exp['actual_yield'] for exp in self.experiment_history])
        else:
            return 0.0
    
    def _select_next_candidate(self, exploration_prob=0.1):
        """獲得関数に基づいて次の候補を選択"""
        # 未試行の組み合わせを取得
        untried_combinations = [
            combo for combo in self.valid_combinations 
            if combo not in self.tried_combinations
        ]
        
        if not untried_combinations:
            print("全ての組み合わせを試行済みです。")
            return None
        
        # 初期の数試行はランダムサンプリング
        if len(self.experiment_history) < 3:
            combo = np.random.choice(len(untried_combinations))
            selected_combo = untried_combinations[combo]
            print(f"🎲 Random sampling: {selected_combo}")
            return selected_combo
        
        # 獲得関数を計算
        best_yield = self._get_best_observed_yield()
        acquisition_scores = []
        
        print("獲得関数値を計算中...")
        for reactant, reagent in untried_combinations:
            product = self.product_dict[(reactant, reagent)]
            reaction_smiles = f"REACTANT:{reactant}REAGENT:{reagent}PRODUCT:{product}"
            
            try:
                pred_mean, pred_variance = self._predict_yield_with_uncertainty(reaction_smiles)
                acq_value = self._acquisition_function(pred_mean, pred_variance, best_yield)
                acquisition_scores.append({
                    'combination': (reactant, reagent),
                    'acquisition_value': acq_value,
                    'predicted_mean': pred_mean,
                    'predicted_variance': pred_variance
                })
            except Exception as e:
                print(f"❌ Error predicting {reactant} + {reagent}: {e}")
                continue
        
        if not acquisition_scores:
            # 獲得関数が計算できない場合はランダム選択
            combo = np.random.choice(len(untried_combinations))
            return untried_combinations[combo]
        
        # 獲得関数値でソート
        acquisition_scores.sort(key=lambda x: x['acquisition_value'], reverse=True)
        
        # ε-greedyの探索戦略
        if np.random.random() < exploration_prob:
            # 探索: ランダム選択
            selected = np.random.choice(acquisition_scores)
            print(f"🎲 Exploration: {selected['combination']} (EI: {selected['acquisition_value']:.4f})")
        else:
            # 活用: 最高獲得関数値を選択
            selected = acquisition_scores[0]
            print(f"🎯 Exploitation: {selected['combination']} (EI: {selected['acquisition_value']:.4f})")
        
        return selected['combination']
    
    def _evaluate_candidate(self, reactant, reagent, trial_num):
        """候補を評価（実験を実行）"""
        product = self.product_dict[(reactant, reagent)]
        reaction_smiles = f"REACTANT:{reactant}REAGENT:{reagent}PRODUCT:{product}"
        
        # MC Dropoutによる予測
        try:
            pred_mean, pred_variance = self._predict_yield_with_uncertainty(reaction_smiles)
        except Exception as e:
            print(f"❌ Prediction error: {e}")
            return None
        
        # 実際の収率を取得
        key = (reactant, reagent, product)
        actual_yield = self.true_yield_dict.get(key)
        
        if actual_yield is None:
            print(f"❗ No ground truth for: {reactant} + {reagent} → {product}")
            actual_yield_pct = 0.0
        else:
            actual_yield_pct = actual_yield * 100  # パーセント変換
        
        # 獲得関数の計算
        best_yield = self._get_best_observed_yield()
        acquisition_value = self._acquisition_function(pred_mean, pred_variance, best_yield)
        
        # 誤差の計算
        error_pct = pred_mean - actual_yield_pct if actual_yield is not None else None
        
        # 実験結果を記録
        experiment_result = {
            'trial': trial_num,
            'reactant': reactant,
            'reagent': reagent,
            'product': product,
            'reaction_smiles': reaction_smiles,
            'predicted_mean': pred_mean,
            'predicted_variance': pred_variance,
            'actual_yield': actual_yield_pct,
            'error_pct': error_pct,
            'acquisition_value': acquisition_value
        }
        
        # 出力
        print(f"🔎 Trial {trial_num}: {reactant} + {reagent} → {product}")
        print(f"   📈 Predicted: {pred_mean:.2f}% ± {np.sqrt(pred_variance):.2f}%")
        print(f"   🧪 Ground truth: {actual_yield_pct:.2f}%" if actual_yield is not None else "   🧪 Ground truth: None")
        if error_pct is not None:
            print(f"   ❗ Error: {error_pct:+.2f}%")
        print(f"   🎯 Acquisition: {acquisition_value:.4f}")
        
        return experiment_result
    
    def optimize(self, n_trials=50):
        """ベイズ最適化の実行"""
        print(f"ベイズ最適化を開始 ({n_trials}試行)")
        print("="*60)
        
        for trial in range(1, n_trials + 1):
            # 次の候補を選択
            candidate = self._select_next_candidate()
            if candidate is None:
                print("全ての組み合わせを試行完了。")
                break
            
            reactant, reagent = candidate
            
            # 候補を評価
            result = self._evaluate_candidate(reactant, reagent, trial)
            if result is None:
                continue
            
            # 結果を記録
            self.experiment_history.append(result)
            self.tried_combinations.add((reactant, reagent))
            
            # 進捗表示
            current_best = max([exp['actual_yield'] for exp in self.experiment_history])
            print(f"   💡 Current best: {current_best:.2f}%")
            print("-" * 60)
        
        print(f"\n最適化完了! 総試行数: {len(self.experiment_history)}")
        
        # 最終結果
        if self.experiment_history:
            best_exp = max(self.experiment_history, key=lambda x: x['actual_yield'])
            print(f"🏆 最高収率: {best_exp['actual_yield']:.2f}%")
            print(f"🏆 最適組み合わせ: {best_exp['reactant']} + {best_exp['reagent']} → {best_exp['product']}")
            return best_exp
        else:
            return None
    
    def get_best_experiments(self, top_k=5):
        """上位k個の実験結果を取得"""
        sorted_experiments = sorted(
            self.experiment_history, 
            key=lambda x: x['actual_yield'], 
            reverse=True
        )
        return sorted_experiments[:top_k]
    
    def get_optimization_summary(self):
        """最適化の概要統計を取得"""
        if not self.experiment_history:
            return {}
        
        actual_yields = [exp['actual_yield'] for exp in self.experiment_history]
        predicted_means = [exp['predicted_mean'] for exp in self.experiment_history]
        errors = [exp['error_pct'] for exp in self.experiment_history if exp['error_pct'] is not None]
        
        return {
            'total_trials': len(self.experiment_history),
            'max_yield': max(actual_yields),
            'mean_yield': np.mean(actual_yields),
            'std_yield': np.std(actual_yields),
            'mae_error': np.mean(np.abs(errors)) if errors else None,
            'rmse_error': np.sqrt(np.mean(np.array(errors)**2)) if errors else None,
            'coverage': len(self.tried_combinations) / len(self.valid_combinations) * 100
        }

In [16]:
# 辞書の作成
reactant_list, reagent_list, product_list, product_dict, true_yield_dict = create_reaction_dictionaries(df.head(1000))

# ベイズ最適化器を初期化（新しいインターフェース）
optimizer = BayesianOptimizationWithMCDropout(
    model=yield_model,
    tokenizer=yield_tokenizer,
    reactant_list=reactant_list,
    reagent_list=reagent_list,
    product_dict=product_dict,
    true_yield_dict=true_yield_dict,
    n_mc_samples=30
)

Reactant candidates: 25
Reagent candidates: 23
Product candidates: 23
Known combinations: 543
有効な組み合わせ数: 543


In [ ]:
# ベイズ最適化の実行
best_result = optimizer.optimize(n_trials=30)

# 最適化の概要統計を表示
summary = optimizer.get_optimization_summary()
print(f"\n=== 最適化概要 ===")
print(f"総試行数: {summary.get('total_trials', 0)}")
print(f"最高収率: {summary.get('max_yield', 0):.2f}%")
print(f"平均収率: {summary.get('mean_yield', 0):.2f}%")
print(f"収率標準偏差: {summary.get('std_yield', 0):.2f}%")
if summary.get('mae_error'):
    print(f"予測誤差 (MAE): {summary['mae_error']:.2f}%")
    print(f"予測誤差 (RMSE): {summary['rmse_error']:.2f}%")
print(f"探索範囲: {summary.get('coverage', 0):.1f}%")

ベイズ最適化を開始 (30試行)
🎲 Random sampling: ('COc1cc(Cl)ccc1F', 'CN(C)c1ccccc1-c1ccccc1P(c1ccccc1)c1ccccc1.OB(O)B(O)O')
🔎 Trial 1: COc1cc(Cl)ccc1F + CN(C)c1ccccc1-c1ccccc1P(c1ccccc1)c1ccccc1.OB(O)B(O)O → COc1cc(B(O)O)ccc1F
   📈 Predicted: 71.69% ± 1.71%
   🧪 Ground truth: 30.78%
   ❗ Error: +40.91%
   🎯 Acquisition: 71.6821
   💡 Current best: 30.78%
------------------------------------------------------------
🎲 Random sampling: ('Clc1ccc2[nH]ccc2c1', 'c1ccc(P(c2ccccc2)C2CCCCC2)cc1.OB(O)B(O)O')
🔎 Trial 2: Clc1ccc2[nH]ccc2c1 + c1ccc(P(c2ccccc2)C2CCCCC2)cc1.OB(O)B(O)O → OB(O)c1ccc2[nH]ccc2c1
   📈 Predicted: 73.39% ± 1.29%
   🧪 Ground truth: 46.13%
   ❗ Error: +27.26%
   🎯 Acquisition: 42.6040
   💡 Current best: 46.13%
------------------------------------------------------------
🎲 Random sampling: ('Brc1ccc2c(c1)OCO2', 'c1ccc(-c2ccccc2P(C2CCCCC2)C2CCCCC2)cc1.OB(O)B(O)O')
🔎 Trial 3: Brc1ccc2c(c1)OCO2 + c1ccc(-c2ccccc2P(C2CCCCC2)C2CCCCC2)cc1.OB(O)B(O)O → OB(O)c1ccc2c(c1)OCO2
   📈 Predicted: 72.39% ±

## 可視化

In [ ]:
def visualize_optimization_results(experiment_history, out_dir=None, show=True, dpi=180):
    """
    最適化結果の可視化（greedy_optuna.ipynbのvisualize_logs関数を参考）
    
    Args:
        experiment_history: 実験履歴のリスト
        out_dir: 出力ディレクトリ
        show: グラフを表示するかどうか
        dpi: 解像度
    """
    import os
    
    if not experiment_history:
        print("実験データがありません。")
        return
    
    # 出力ディレクトリの準備
    if out_dir:
        save_dir = os.path.join(out_dir, "visualization")
        os.makedirs(save_dir, exist_ok=True)
    else:
        save_dir = None
    
    # データの準備
    trials = [exp['trial'] for exp in experiment_history]
    actual_yields = [exp['actual_yield'] for exp in experiment_history]
    predicted_means = [exp['predicted_mean'] for exp in experiment_history]
    predicted_stds = [np.sqrt(exp['predicted_variance']) for exp in experiment_history]
    errors = [exp['error_pct'] for exp in experiment_history if exp['error_pct'] is not None]
    
    # 累積最大値を計算
    cumulative_max = np.maximum.accumulate(actual_yields)
    
    # 全体指標の計算
    if len(actual_yields) > 0 and len(errors) > 0:
        mae_pct = np.mean(np.abs(errors))
        rmse_pct = np.sqrt(np.mean(np.array(errors)**2))
        bias_pct = np.mean(errors)
        
        print(f"=== 全体指標 ===")
        print(f"試行数: {len(experiment_history)}")
        print(f"MAE: {mae_pct:.2f}%")
        print(f"RMSE: {rmse_pct:.2f}%")
        print(f"Bias: {bias_pct:+.2f}%")
        print(f"最高収率: {max(actual_yields):.2f}%")
    
    # 図の作成
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. パリティプロット（予測 vs 実測）
    ax = axes[0, 0]
    ax.errorbar(predicted_means, actual_yields, xerr=predicted_stds, 
               fmt='o', alpha=0.6, capsize=3)
    ax.plot([0, 100], [0, 100], 'r--', label='Perfect Prediction')
    ax.set_xlabel('Predicted Yield (%)')
    ax.set_ylabel('Actual Yield (%)')
    ax.set_title('Parity: Prediction vs Truth')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. 最適化の進行
    ax = axes[0, 1]
    ax.plot(trials, actual_yields, 'o-', alpha=0.6, label='Actual Yield', markersize=4)
    ax.plot(trials, cumulative_max, 'r-', linewidth=2, label='Best So Far')
    ax.set_xlabel('Trial')
    ax.set_ylabel('Yield (%)')
    ax.set_title('Optimization Progress')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 3. 不確実性 vs 実測収率
    ax = axes[0, 2]
    ax.scatter(predicted_stds, actual_yields, alpha=0.6)
    ax.set_xlabel('Prediction Uncertainty (Std)')
    ax.set_ylabel('Actual Yield (%)')
    ax.set_title('Uncertainty vs Actual Yield')
    ax.grid(True, alpha=0.3)
    
    # 4. 誤差ヒストグラム
    ax = axes[1, 0]
    if errors:
        ax.hist(errors, bins=20, alpha=0.7, edgecolor='black')
        ax.axvline(0, color='red', linestyle='--', label='Perfect Prediction')
        ax.set_xlabel('Prediction Error (pred - true) [%]')
        ax.set_ylabel('Count')
        ax.set_title('Error Histogram')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # 5. 予測収率の分布
    ax = axes[1, 1]
    ax.hist(predicted_means, bins=20, alpha=0.5, label='Predicted', edgecolor='black')
    ax.hist(actual_yields, bins=20, alpha=0.5, label='Actual', edgecolor='black')
    ax.set_xlabel('Yield (%)')
    ax.set_ylabel('Count')
    ax.set_title('Yield Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 6. 収率 vs 試行番号（散布図）
    ax = axes[1, 2]
    ax.scatter(trials, predicted_means, alpha=0.5, label='Predicted', s=20)
    ax.scatter(trials, actual_yields, alpha=0.5, label='Actual', s=20)
    ax.set_xlabel('Trial')
    ax.set_ylabel('Yield (%)')
    ax.set_title('Yields over Trials')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 保存
    if save_dir:
        fig.savefig(os.path.join(save_dir, "optimization_results.png"), 
                   dpi=dpi, bbox_inches="tight")
        print(f"図を保存しました: {save_dir}/optimization_results.png")
    
    if show:
        plt.show()
    
    plt.close(fig)
    
    return {
        'mae_pct': mae_pct if errors else None,
        'rmse_pct': rmse_pct if errors else None,
        'bias_pct': bias_pct if errors else None,
        'max_yield': max(actual_yields) if actual_yields else None,
        'save_dir': save_dir
    }

In [ ]:
# 結果の可視化（新しいプロット関数を使用）
results = visualize_optimization_results(optimizer.experiment_history, show=True)

# 上位5つの実験結果を表示
print("\n=== 上位5つの実験結果 ===")
best_experiments = optimizer.get_best_experiments(top_k=5)
for i, exp in enumerate(best_experiments, 1):
    print(f"\n{i}位: 試行{exp['trial'] + 1}")
    print(f"  Reactant: {exp['reactant']}")
    print(f"  Reagent: {exp['reagent']}")
    print(f"  Product: {exp['product']}")
    print(f"  実際の収率: {exp['actual_yield']:.2f}%")
    print(f"  予測収率: {exp['predicted_mean']:.2f}% ± {np.sqrt(exp['predicted_variance']):.2f}%")
    if exp['error_pct'] is not None:
        print(f"  誤差: {exp['error_pct']:+.2f}%")
    print(f"  獲得関数値: {exp['acquisition_value']:.4f}")

# reactant-reagent組み合わせの統計情報
print("\n=== Reactant-Reagent組み合わせ統計 ===")
combo_stats = optimizer.get_combination_statistics()
print(f"探索された組み合わせ数: {len(combo_stats)}")

# 最も多く選ばれた組み合わせ上位3つ
combo_counts = {combo: len(data) for combo, data in combo_stats.items()}
top_combos = sorted(combo_counts.items(), key=lambda x: x[1], reverse=True)[:3]

print("\n最も多く探索された組み合わせ:")
for i, ((reactant, reagent), count) in enumerate(top_combos, 1):
    avg_yield = np.mean([d['actual_yield'] for d in combo_stats[(reactant, reagent)]])
    print(f"  {i}. {reactant} + {reagent}: {count}回, 平均収率: {avg_yield:.2f}%")